In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'head_diameter'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['image_url']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

2527


In [12]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [29]:
df[df['HeadDiameter'].astype(str)!='nan']['HeadDiameter'].explode().value_counts()

20mm      75
13mm      57
18mm      45
15mm      34
16mm      32
14mm      20
12mm      15
10mm      14
17mm      13
30mm      10
25mm       6
22mm       6
19mm       6
9mm        6
11mm       5
12.5mm     4
21mm       3
8mm        2
6.5mm      2
9.5mm      2
23mm       1
13.5mm     1
32mm       1
7.5mm      1
11.5mm     1
26mm       1
6.3mm      1
25/64"     1
36.5mm     1
16.5mm     1
38mm       1
24mm       1
17.5mm     1
28mm       1
8.5mm      1
Name: HeadDiameter, dtype: int64

In [25]:
df[curation_col]='n/a'
matchna=df[['external_id',curation_col]]

In [20]:
pat_mm=rf'''((?<=\d)\s?\-?mm)|()'''         
df['mm_name'] = df['product_name'].apply(lambda x: re_extract(pat_mm, str(x)))
df['mm_ld'] = df['long_desc'].apply(lambda x: re_extract(pat_mm, str(x)))
df['mm_hd'] = df['HeadDiameter'].apply(lambda x: re_extract(pat_mm, str(x)))

pat_head=r'(?i)(?i)(Head Diameter)|()'
df['head']=df['long_desc'].apply(lambda x: re_extract(pat_head, x))

pat_hd=r'(?i)(\#\d)|()'
df['hd']=df['long_desc'].apply(lambda x: re_extract(pat_hd, x))

In [26]:
df[df['hd'].astype(str)!='[]'].head(5)#['hd'].explode().value_counts()

buckets external_id                         product_name  \
209  Fixings & Fasteners       12694  5/32 Trunk Lock Swivel Emblem Rivet   
282  Fixings & Fasteners      131328                  Headlight Nylon Nut   
288  Fixings & Fasteners      131433          SEMS Hex Head Tapping Screw   
299  Fixings & Fasteners       13874                  Door Trim Panel Nut   
387  Fixings & Fasteners       12109               Chrysler Retaining Nut   

                                                                                                                                                                                                                                        long_desc  \
209  Countersunk head, closed-end automotive rivet. Aluminum Rivet/Steel Mandrel. Used for trunk locks. 5/32” diameter, 5/16" flange diameter. Grip range: 1/8" - 3/16". Replaces: GM 9432137, Chrysler 6032138. Requires KM #48-2909 Nose Piece.   
282                                                                                                        Screw Size M5.5 (#12) Head Size: 10mm Fits into 10.5mm x 11mm Hole Jeep Grand Cherokee & Commander 2005 - On Replaces Mopar 6508072-AA   
288                                          M4.8-1.61 x 32mm (#10 x 1-1/4") Hex Head Sems® Tapping Screw Phosphate Washer Outside Diameter: 19mm (3/4") Hex: 8mm Chrysler Town & Country and Ram 2009 - 2014 Replaces Chrysler 6509539-AA Pkg 25   
299                                                                                                                            M4.2 (#8) Screw Size Head Size: 16mm x 16mm Stem Length: 10mm Caravan, Town & Country And Voyager Replaces 6035401   
387                                                                                                                                    Chrysler Push-in Fasteners Chry Door Panel Metal Retainer Nut #6. LHS ‘93-on. Repl 6034336/6034812/6035807   

    keywords packageweight packagequantity  \
209                    0.1              25   
282                    0.1              50   
288                    0.4              25   
299                    0.1              25   
387                    0.1              25   

                                                                                    shortdescription  \
209                                  <p>Aluminum Rivet/Steel Mandrel GM 9432137/Chrysler 6032138</p>   
282                                                                               <p>Black Nylon</p>   
288                                                                       <p>Chrysler 6509539-AA</p>   
299                                                                               <p>Green Nylon</p>   
387  <p>Chrysler Door Panel Metal Retainer Nut #6. LHS '93-On. Replaces: 6034336/6034812/6035807</p>   

           Type _ofPieces PrivateLabel Finish    HeadStyle DriveRecess  \
209  Closed End       NaN           No    NaN  Countersunk         NaN   
282         NaN        50           No    NaN          NaN         NaN   
288         NaN        25           No    NaN          NaN         NaN   
299         NaN       NaN           No    NaN          NaN         NaN   
387         NaN       NaN           No    NaN          NaN         NaN   

    Material Grade Length Diameter ThreadType ThreadPitch  \
209      NaN   NaN    NaN    5/32"        NaN         NaN   
282      NaN   NaN    NaN      NaN        NaN         NaN   
288      NaN   NaN    NaN      NaN        NaN         NaN   
299      NaN   NaN    NaN      NaN        NaN         NaN   
387      NaN   NaN    NaN      NaN        NaN         NaN   

    TensileStrength_PSI_ Color Application BandWidth ScrewSize HeadHeight  \
209                  NaN   NaN         NaN       NaN       NaN        NaN   
282                  NaN   NaN         NaN       NaN       NaN        NaN   
288                  NaN   NaN         NaN       NaN       NaN        NaN   
299                  NaN   NaN         NaN       NaN       Na

In [27]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchna))


# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchna


In [12]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [28]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [14]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute, df, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
#     matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
# looks_good('Kimball MidWest', attribute, df, matchfinal) 

In [15]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute, df, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
#     matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
# looks_good('Kimball MidWest', attribute, df, matchna) 

In [16]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute, df, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
#     matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
# looks_good('Kimball MidWest', attribute, df, matchex) 